In [1]:
!pip install contractions num2words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.3 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
from torch.nn.functional import normalize as l2_norm

import random
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.metrics import bleu_score
from pprint import pprint

device = "cuda:0" if torch.cuda.is_available() and torch.cuda.device_count() > 1 else "cuda" if torch.cuda.is_available() else "cpu"
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

in_dir = '/kaggle/input'
out_dir = '/kaggle/working'
temp_dir = '/kaggle/temp'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-05-08 16:02:08.955054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 16:02:08.955151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 16:02:09.100514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/combined-lince-and-cmu-dog/test_combined.csv
/kaggle/input/combined-lince-and-cmu-dog/val_combined.csv
/kaggle/input/combined-lince-and-cmu-dog/train_combined.csv


In [3]:
data = {}
train_data, dev_data, test_data = pd.read_csv(in_dir + '/combined-lince-and-cmu-dog/train_combined.csv'), pd.read_csv(in_dir + '/combined-lince-and-cmu-dog/val_combined.csv'), pd.read_csv(in_dir + '/combined-lince-and-cmu-dog/test_combined.csv')
data['X_train'], data['y_train'] = train_data['eng'].values, train_data['hing'].values
data['X_dev'], data['y_dev'] = dev_data['eng'].values, dev_data['hing'].values
data['X_test'], data['y_test'] = test_data['eng'].values, test_data['hing'].values

In [4]:
import string
import contractions
from num2words import num2words

import nltk
# nltk.download('punkt')

from nltk.tokenize import word_tokenize

from torchtext.vocab import build_vocab_from_iterator, Vocab

SOS = '<s>'
EOS = '</s>'
UNK = '<UNK>'
PAD = '<PAD>'

def convert_num2words(sent):
    words = sent.split()
    result_words = []
    for word in words:
        if word.isdigit():
            result_words.append(num2words(int(word)))
        else:
            result_words.append(word)
    return ' '.join(result_words)

def remove_punctuation(sent):
    translator = str.maketrans('', '', string.punctuation)
    return sent.translate(translator)

def pre_process(sent):
    sent = contractions.fix(sent)
    sent = sent.lower()
    sent = convert_num2words(sent)
    sent = remove_punctuation(sent)
    sent = word_tokenize(sent)
    return sent

l = 1
data['train_tokens'] = [pre_process(i) for i in data['X_train'][ :: l]], [pre_process(i) for i in data['y_train'][ :: l]]
data['dev_tokens'] = [pre_process(i) for i in data['X_dev'][ :: l]], [pre_process(i) for i in data['y_dev'][ :: l]]
data['test_tokens'] = [pre_process(i) for i in data['X_test'][ :: l]], [pre_process(i) for i in data['y_test'][ :: l]]

eng_vocab = build_vocab_from_iterator(data['train_tokens'][0], specials=[UNK, PAD, SOS, EOS], min_freq=3)
eng_vocab.set_default_index(eng_vocab[UNK])

hing_vocab = build_vocab_from_iterator(data['train_tokens'][1], specials=[UNK, PAD, SOS, EOS], min_freq=3)
hing_vocab.set_default_index(hing_vocab[UNK])

torch.save(eng_vocab, out_dir + '/eng_vocab.pt')
torch.save(hing_vocab, out_dir + '/hing_vocab.pt')

In [5]:
def pad_collate(batch):
  (xx, yy) = zip(*batch)
  
  xx = [torch.tensor([eng_vocab[SOS]] + [eng_vocab[token] for token in x] + [eng_vocab[EOS]]) for x in xx]
  yy = [torch.tensor([hing_vocab[SOS]] + [hing_vocab[token] for token in y] + [hing_vocab[EOS]]) for y in yy]

  x_pad = pad_sequence(xx, padding_value=eng_vocab[PAD])
  y_pad = pad_sequence(yy, padding_value=hing_vocab[PAD])

  return x_pad.to(device), y_pad.to(device)
  

train_data = list(zip(data['train_tokens'][0], data['train_tokens'][1]))
dev_data = list(zip(data['dev_tokens'][0], data['dev_tokens'][1]))
test_data = list(zip(data['test_tokens'][0], data['test_tokens'][1]))

batch_size = 32

loaders = {
      'train': DataLoader(train_data, batch_size=batch_size, collate_fn=pad_collate, shuffle=False),
      'dev': DataLoader(dev_data, batch_size=batch_size, collate_fn=pad_collate, shuffle=True),
      'test': DataLoader(test_data, batch_size=batch_size, collate_fn=pad_collate, shuffle=True),
    }

In [6]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.energy_layer = nn.Linear(hidden_size, 1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch size, hidden size], encoder_outputs: [seq len, batch size, hidden size]
        hidden = hidden.permute(0, 1, 2)
        hidden = hidden.transpose(0, 1)
        encoder_outputs = encoder_outputs.transpose(0, 1)
#         print(f"Query Shape: {hidden.size()}")
#         print(f"Encoder Outputs Shape: {encoder_outputs.size()}")
        query_layer = self.query_layer(hidden)
#         print(f"Query Layer Shape: {query_layer.size()}")
        key_layer = self.key_layer(encoder_outputs)
#         print(f"Key Layer Shape: {key_layer.size()}")
        energy = self.energy_layer(torch.tanh(query_layer + key_layer))
#         print(f"Scores Shape: {energy.size()}")
        attention = energy.squeeze(2).unsqueeze(1)
#         print(f"Scores Shape: {energy.size()}")

        return self.softmax(attention)

class LuongAttention(nn.Module):
    def __init__(self, hidden_size):
        super(LuongAttention, self).__init__()
        self.softmax = nn.Softmax(dim=0)

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch size, hidden size], encoder_outputs: [seq len, batch size, hidden size]
        attention = torch.bmm(encoder_outputs.transpose(0, 1), hidden.unsqueeze(2)).squeeze(2)
        return self.softmax(attention.transpose(0, 1))

In [18]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return outputs, hidden_state, cell_state

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(3801, 128)
  (LSTM): LSTM(128, 128, dropout=0.5)
)


In [19]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size, attn=False):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    if attn:
        embedding_size = embedding_size + hidden_size
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)
    
    self.attn = attn
    if attn:
        self.attention = BahdanauAttention(hidden_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state, encoder_outputs):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    lstm_input = self.dropout(self.embedding(x))
#     print(f"Embeddings Shape: {embedding.size()}")
#     print(f"Hidden Shape: {hidden_state.size()}")
    if self.attn:
        attention_weights = self.attention(hidden_state, encoder_outputs)
    #     print(f"Weights Shape: {attention_weights.size()}")
        context = torch.bmm(attention_weights, encoder_outputs.transpose(0, 1))
        context = context.transpose(0, 1)
    #     print(f"Context Shape: {context.size()}")
        lstm_input = torch.cat((lstm_input, context), dim=2)
#     print(f"Input LSTM Shape: {lstm_input.size()}")
    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(lstm_input, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4776, 128)
  (LSTM): LSTM(128, 128, dropout=0.5)
  (fc): Linear(in_features=128, out_features=4776, bias=True)
)


In [20]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(hing_vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    encoder_outputs, hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state, encoder_outputs)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs

In [29]:
def translate_sentence(model, sentence, eng_vocab, hing_vocab, device, max_length=50):
    tokens = pre_process(sentence) if type(sentence) == str else sentence.copy()
    
    tokens.insert(0, SOS)
    tokens.append(EOS)
    text_to_indices = [eng_vocab[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        encoder_outputs, hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [hing_vocab[SOS]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell, encoder_outputs)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == hing_vocab[EOS]:
            break

    translated_sentence = [hing_vocab.get_itos()[idx] for idx in outputs]
    return translated_sentence[1:]

weights = [(1, 0, 0, 0), 
           (0.5, 0.5), 
           (0.33, 0.33, 0.33, 0), 
           (0.25, 0.25, 0.25, 0.25),
           (0.2, 0.2, 0.2, 0.2, 0.2),
           (0.16, 0.16, 0.16, 0.16, 0.16, 0.16),
           (0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14),
           (0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125)]

def bleu_score(refs, candidates):
  score = [0] * len(weights)
  for i in range(len(refs)):
    for j, w in enumerate(weights):
      score[j] += sentence_bleu([refs[i]], candidates[i], weights=w)

  for i in range(len(score)):
    score[i] = score[i] / len(refs)
    score[i] = round(score[i], 6)
  
  return sum(score) / len(score)

def bleu(data, model, eng_vocab, hing_vocab, device):
    targets = []
    outputs = []

    for example in data:
        src = example[0]
        trg = example[1]

        prediction = translate_sentence(model, src, eng_vocab, hing_vocab, device)
        prediction = prediction[:-1]  # remove <eos> token
        
        targets.append(trg)
        outputs.append(prediction)
#         print(f"English: {' '.join(src)}")
#         print(f"Hinglish: {' '.join(trg)}")
#         print(f"Translated: {' '.join(outputs[-1])}")
#         print('\n')

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, f'{out_dir}/checkpoint-LSTM')
    torch.save(model.state_dict(),f'{out_dir}/checkpoint-LSTM-SD')

In [59]:
def train(lr=0.001, dropout=0.5, hidden_size=128, embedding_size=128, attn=False):
    # Hyperparameters
    num_epochs = 15
    input_size_encoder = len(eng_vocab)
    encoder_embedding_size = embedding_size
    hidden_size = hidden_size
    num_layers = 1
    encoder_dropout = dropout

    encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                               hidden_size, num_layers, encoder_dropout).to(device)
    
    input_size_decoder = len(hing_vocab)
    decoder_embedding_size = embedding_size
    hidden_size = hidden_size
    decoder_dropout = dropout
    output_size = len(hing_vocab)
    attn = True
    decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size, attn).to(device)
    
    learning_rate = lr
    writer = SummaryWriter(f"{out_dir}/runs/lstm_loss_plot")
    step = 0

    model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    pad_idx = hing_vocab[PAD]
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

    best_loss = 999999
    best_epoch = -1
    sentence = "The music in movie is very catchy and enjoyable, so I agree."
    ts  = []

    for epoch in range(num_epochs):
      print("Epoch - {} / {}".format(epoch + 1, num_epochs))
      model.eval()
      translated_sentence = translate_sentence(model, sentence, eng_vocab, hing_vocab, device, max_length=50)
      print(f"Translated example sentence: \n {translated_sentence}")
      ts.append(translated_sentence)

      model.train()
      epoch_loss = 0.0
      for batch_idx, (input, target) in enumerate(loaders['train']):
        # Pass the input and target for model's forward method
        output = model(input, target)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Calculate the loss value for every epoch
        loss = criterion(output, target)

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Update the weights values using the gradients we calculated using bp 
        optimizer.step()
        step += 1
        epoch_loss += loss.item()
        writer.add_scalar("Training loss", loss, global_step=step)    

      if epoch_loss < best_loss:
        best_loss = epoch_loss
        best_epoch = epoch
        checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
        if ((epoch - best_epoch) >= 10):
          print("no improvement in 10 epochs, break")
          break
      print("Epoch_Loss - {}".format(epoch_loss / len(loaders['train'])))
      print()

    print(epoch_loss / len(loaders['train']))

    score = bleu(dev_data, model, eng_vocab, hing_vocab, device)
#     details = {
#         'learning_rate': lr,
#         'dropout': dropout,
#         'hidden_size': hidden_size,
#         'embedding_size': embedding_size,
#         'attention': attn,
#         'bleu_score': round(score, 4)
#     }
#     scores.append(details)
    print(f"Bleu score {score * 100:.2f}")
    print("============================================================")
    return model

In [57]:
params = {
    'attn': [False, True],
    'lr': [0.001, 0.01, 0.1],
    'dropout': [0, 0.5, 1],
    'hidd_size': [64, 128, 256],
    'embed_size': [64, 128, 256],
}

# scores = []
for attn in params['attn']:
    for lr in params['lr']:
        for drp in params['dropout']:
            for hidd in params['hidd_size']:
                    train(lr, drp, hidd, hidd, attn)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch - 1 / 15
Translated example sentence: 
 ['regarding', 'dilata', 'just', 'effect', 'determine', 'liam', 'people', 'upayog', 'mujhey', 'piece', 'common', 'mujhey', 'piece', 'life', 'life', 'gudamainhil', 'kisko', 'sangeetamay', 'sally', 'lagu', 'early', 'entertainment', 'chahoge', 'unn', 'sally', 'lagu', 'bahot', 'common', '5810', 'haiha', 'dusra', 'chicago', 'ratings', 'ship', 'subh', 'nikli', 'intrinsic', 'intrinsic', 'gyi', 'entertainment', 'namaskar', 'namaskar', 'bht', 'nirdeshak', 'nirdeshak', 'nirdeshak', 'ata', 'unexpected', 'rehna', 'jisake']
saving

Epoch_Loss - 6.303222230502537

Epoch - 2 / 15
Translated example sentence: 
 ['kya', '</s>']
saving

Epoch_Loss - 5.992057704736316

Epoch - 3 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', 'hai', 'ki', '<UNK>', '<UNK>', 'hai', '</s>']
saving

Epoch_Loss - 5.839967604667422

Epoch - 4 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', '<UNK>', 'hai', 'ki', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '</s>']


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bleu score 27.31
Epoch - 1 / 15
Translated example sentence: 
 ['badhiya', 'trigger', 'cute', 'a', 'charles', 'experience', 'secretary', 'percent', 'color', 'george', 'omg', 'imdb', 'baltimore', 'baltimore', 'disqualify', 'obviously', 'george', 'kahin', 'anyatha', 'sweet', 'carl', 'jaaree', 'any', 'kiddy', 'historical', 'aisa', 'fear', 'koee', 'hole', 'gyaat', 'island', 'next', 'space', 'gyaat', 'suppose', 'scientist', 'lines', 'lete', 'personally', 'george', 'george', 'kahin', 'anyatha', 'sweet', 'carl', 'jaaree', 'any', 'kiddy', 'historical', 'aisa']
saving

Epoch_Loss - 6.212055610285865

Epoch - 2 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', 'hai', '</s>']
saving

Epoch_Loss - 5.810341250328791

Epoch - 3 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '</s>']
saving

Epoch_Loss - 5.678671456518627

Epoch - 4 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 5-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bleu score 28.96
Epoch - 1 / 15
Translated example sentence: 
 ['koi', 'vas', 'lad', 'karte', 'princes', 'walker', 'dawn', 'doge', 'dam', 'robbers', 'mahal', 'rakhte', 'dubara', 'lad', 'karte', 'karte', 'tippanee', 'bacche', 'bacche', 'intrinsically', 'karne', 'princes', 'aap', 'deya', 'celine', 'age', 'village', 'thriller', 'lots', 'cofounder', 'hainks', 'eh', 'chali', 'force', 'melt', 'min', 'budget', 'batati', 'cuteness', 'imitation', 'jack', 'mi', 'ache', 'fitness', 'founder', 'probation', 'avengers', 'liking', 'light', 'usme']
saving

Epoch_Loss - 6.0459660367360195

Epoch - 2 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '</s>']
saving

Epoch_Loss - 5.65329650186357

Epoch - 3 / 15
Translated example sentence: 
 ['mujhe', 'lagta', 'hai', 'ki', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '</s>']
saving

Epoch_Loss - 5.4712304671605425

Epoch - 4 / 15
Translated example sentence: 
 ['mujhe', 'l

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 6-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bleu score 34.24
Epoch - 1 / 15
Translated example sentence: 
 ['kehna', 'creatures', 'atyant', 'inspired', 'bhul', 'inspired', 'fight', 'weekend', 'meaningless', 'kay', 'singers', 'thought', 'easier', 'easier', 'than', 'government', 'studios', 'banaaya', 'fbi', 'uski', 'ne', 'mordern', 'konse', 'hawo', 'nineteen', 'karega', 'hein', 'parwaah', 'pyaara', 'dekhoge', 'eighteen', 'ratai', 'ratai', 'bar', 'bar', 'aapka', 'vahan', 'immortality', 'mani', 'vishwas', 'combat', 'yeahuske', 'police', 'kisakee', 'pasad', 'pasad', 'thi', 'pehle', 'explain', 'skill']
saving

Epoch_Loss - 6.081113867343418

Epoch - 2 / 15
Translated example sentence: 
 ['kya', '</s>']
saving

Epoch_Loss - 5.718144431000664

Epoch - 3 / 15
Translated example sentence: 
 ['kya', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '</s>']
saving

Epoch_Loss - 5.554939590749287

Epoch - 4 / 15
Translated example sentence: 
 ['mujhe', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK

In [79]:
pd.DataFrame(sorted(scores, key=lambda x: x['bleu_score']))

,learning_rate,dropout,hidden_size,embedding_size,attention,bleu_score
,0.100,0.5,128,128,True,0.061230
,0.100,0.5,64,64,False,0.068300
,0.100,0.5,256,256,False,0.077900
,0.100,0.5,128,128,False,0.079600
,0.100,0.5,64,64,True,0.117270
,0.100,0.5,256,256,True,0.131562
,0.010,0.5,256,256,False,0.198800
,0.010,0.5,128,128,False,0.213600
,0.010,0.5,128,128,True,0.251508
,0.010,0.5,64,64,False,0.270600


In [60]:
model = train(hidden_size=256, embedding_size=256, attn=True)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch - 1 / 15
Translated example sentence: 
 ['muej', 'sipahi', 'bahar', 'bahar', 'bahar', 'real', 'jitana', 'coulson', 'recommend', 'obadiah', 'milate', 'share', 'bhadiya', 'pareshaani', 'is', 'maanana', 'sipahi', 'shikaaree', 'stereotype', 'film', 'sentence', 'turning', 'vipareet', 'warn', 'list', 'gye', 'pichhalee', 'comments', 'thawo', 'ayega', 'ninety', 'gang', 'received', 'abide', 'hang', 'baras', 'underdog', 'ladka', 'latest', 'critic', 'phir', 'vitamin', 'kristen', 'lagegi', 'wwe', 'akela', 'peeche', 'misread', 'magazines', 'theater']
saving

Epoch_Loss - 6.025582270962851

Epoch - 2 / 15
Translated example sentence: 
 ['mujhe', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'hai', '</s>']
saving

Epoch_Loss - 5.588074549796089

Epoch - 3 / 15
Translated example sentence: 
 ['movie', 'movie', 'ke', 'baare', 'mein', 'hai', 'ki', 'movie', 'ke', 'liye', '</s>']
saving

Epoch_Loss - 5.267911211838798

Epoch - 4 / 15
Translated example sentence: 
 ['movie', 'movie',

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 6-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence cont

Bleu score 40.18


In [62]:
sent = translate_sentence(model, 'Have you seen Inception before?', eng_vocab, hing_vocab, device, max_length=50)
' '.join(sent[:-1])

'kya tumne pehle dekha dekhi hai'